# Création du jeu de données

Données recherchées :
- Positions et vélocités des barycentes de système planètaires par rapport au barycentre du système solaire, sur une période la plus longue possible (ça varie d'une planète à l'autre).
- Masse de chaque système planètaire (planète + lunes).

| ID | Name	| Start of Interval	| End of Interval |
|----|------|-------------------|-----------------|
| 10 | Sun [Sol] | B.C. 9999-03-20 00:00 |	A.D. 9999-12-31 12:00 |
| 1 |	Mercury	| B.C. 9999-03-20 00:00	| A.D. 9999-12-31 12:00 |
| 2 |	Venus	| B.C. 9999-03-20 00:00	| A.D. 9999-12-31 12:00 |
| 3 |	Earth	| B.C. 9999-03-20 00:00	| A.D. 9999-12-31 12:00 |
| 4 |	Mars	| 1600-01-01 00:00	| 2600-01-02 00:00 |
| 5 |	Jupiter	| 1600-01-10 00:00 (1600-01-11)	| 2200-01-10 00:00 (2200-01-09) |
| 6 |	Saturn	| 1749-12-30 00:00 (1749-12-31)	| 2250-01-06 00:00 |
| 7 |	Uranus	| 1600-01-04 00:00	| 2399-12-17 00:00 |
| 8 |	Neptune	| 1600-01-04 00:00	| 2400-01-02 00:00 |

## 1. Importation des librairies

In [1]:
import pandas as pd
from astroquery.jplhorizons import Horizons
from pandas import DataFrame
from datetime import datetime
from dateutil.relativedelta import relativedelta

## 2. Définition des planètes cibles et des intervalles de temps pour lesquels récupérer les données

In [2]:
bodies: dict[str, str] = {
    "Soleil": "10",
    "Mercure": "1",
    "Venus":   "2",
    "Terre":   "3",
    "Mars":    "4",
    "Jupiter": "5",
    "Saturne": "6",
    "Uranus":  "7",
    "Neptune": "8",
    #"Pluton": "999",
}

bodies_mass: dict[str, float] = {
    "Soleil": 1.989e30,
    "Mercure": 3.3011e23,
    "Venus":   4.8675e24,
    "Terre":   6.046e24, #Terre + Lune
    "Mars":    6.4171e23, # Mars + Phobos + Deimos
    "Jupiter": 1.8986e27, # Jupiter + Lunes Galiléennes etc. (Augmentation vs 1.8982e27)
    "Saturne": 5.6846e26, # Saturne + Titan etc. (Augmentation vs 5.6834e26)
    "Uranus":  8.6832e25, # Uranus + Lunes (Augmentation vs 8.6810e25)
    "Neptune": 1.0243e26, # Neptune + Triton etc. (Augmentation vs 1.0241e26)
    #"Pluton": 1.303e22
}

date_format: str = "%Y-%m-%d"
min_date: str = "1749-12-31" # Date minimale à partir de laquelle on a des données pour toutes les planètes (Limité par Saturne)
max_date: datetime = datetime.strptime("2200-01-09", date_format) # Date maximale à partir de laquelle on a des données pour toutes les planètes (Limité par Jupiter)
time_step = '1d' # Intervalles de 1 jour

## 3. Récupération des données

In [3]:
all_dataframes = []

for name, target in bodies.items():
    # On va chercher les données par intervalles de 100 ans parce que sur l'intervalle complet, le nombre de lignes retournées dépasse la limite permise par l'API.
    for i in range(5):
        start_date = datetime.strptime(min_date, date_format) + relativedelta(years=i * 100)
        end_date = start_date + relativedelta(years=100, days=-1)
        if end_date > max_date:
            end_date = max_date
        start_date_str = datetime.strftime(start_date, date_format)
        end_date_str = datetime.strftime(end_date, date_format)

        horizons = Horizons(
            id=target,
            location='@0', # Barycentre du système solaire
            epochs={'start': start_date_str, 'stop': end_date_str, 'step': time_step}
        )
        vectors = horizons.vectors()

        df: DataFrame = vectors.to_pandas()

        df['body_id'] = target
        df['body_name'] = name
        df['body_mass'] = bodies_mass[name]

        columns_mask = ['body_id', 'body_name', 'body_mass', 'datetime_jd', 'datetime_str','x','y','z','vx','vy','vz']

        df = df[columns_mask].copy()

        all_dataframes.append(df)

dataset = pd.concat(all_dataframes, ignore_index=True)

dataset.to_json('planetaryBarycenter_coordinates_and_velocities_from_1749-12-31_to_2200-01-09.json', orient='records', lines=True)

print(dataset.sample(5))


       body_id body_name     body_mass  datetime_jd  \
977873       5   Jupiter  1.898600e+27    2516261.5   
159451      10    Soleil  1.989000e+30    2519684.5   
691729       4      Mars  6.417100e+23    2394486.5   
31138       10    Soleil  1.989000e+30    2391371.5   
457607       2     Venus  4.867500e+24    2489102.5   

                          datetime_str         x         y         z  \
977873  A.D. 2177-Mar-10 00:00:00.0000  4.852323  0.943900 -0.112160   
159451  A.D. 2186-Jul-24 00:00:00.0000 -0.001289  0.004288 -0.000049   
691729  A.D. 1843-Oct-13 00:00:00.0000  1.242256 -0.581275 -0.043294   
31138   A.D. 1835-Apr-03 00:00:00.0000  0.000091 -0.002015 -0.000076   
457607  A.D. 2102-Oct-31 00:00:00.0000 -0.415123 -0.585136  0.015671   

              vx            vy            vz  
977873 -0.001538  7.756147e-03  1.208032e-06  
159451 -0.000007 -1.567799e-06  1.311584e-07  
691729  0.006495  1.384617e-02  1.276964e-04  
31138   0.000006 -2.953626e-07 -1.519318e-07  
4